In [2]:
import pandas as pd 
import numpy as np
import dill

import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'vscode+colab' #permette il rendering dei plot sia su vscode che su colab




In [9]:
X_train= pd.read_csv('dataset/X_train_Final.csv')
y_train= pd.read_csv('dataset/y_train.csv')

In [10]:
X_test= pd.read_csv('dataset/X_test_Final.csv')
y_test= pd.read_csv('dataset/y_test.csv')

In [11]:
y_test_id = X_test["id"]

In [12]:
X_train=X_train.drop("Sale_Price", axis=1)
X_test=X_test.drop("Sale_Price", axis=1)

In [13]:
X_train=X_train.drop("id", axis=1)
X_test=X_test.drop("id", axis=1)

In [14]:
y_train=y_train.squeeze() #converte in series
y_test=y_test.squeeze() #converte in series

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from datetime import datetime

In [8]:
results = pd.DataFrame()

In [7]:
results = pd.read_csv('results/prediction.csv')

In [16]:
def model_evaluation(model_name, model ,X_train, y_train, X_test, y_test, log_scale):
    
    if log_scale is True :
        model.fit(X_train, np.log(y_train))
        y_predicted = np.exp(model.predict(X_test))
        
    else:
        model.fit(X_train, np.log(y_train))
        y_predicted = np.exp(model.predict(X_test))
    
    mse = mean_squared_error(y_test, y_predicted)
    rmse = mse**.5
    r2 = r2_score(y_test, y_predicted)
    mae = mean_absolute_error(y_test, y_predicted)
    
    print(f'Model Name : {model_name}')
    print(f'RMSE : {rmse}')
    print(f'R2 : {r2}')
    print(f'MAE : {mae}')
    
    results[model_name] = y_predicted   
    results.to_csv('results/prediction.csv', index=False)

In [17]:
def plot_importance(model):

    feature_names = X_train.columns

    feature_importance_df = pd.DataFrame(model.feature_importances_, feature_names)
    feature_importance_df = feature_importance_df.reset_index()
    feature_importance_df.columns = ['Feature', 'Importance']
    feature_importance_df_top_10 = feature_importance_df.sort_values(by='Importance', ascending=False).head(10)
    fig=px.bar(feature_importance_df,feature_importance_df_top_10.Feature, feature_importance_df_top_10.Importance)
    fig.show()

In [40]:
X_test_temp = pd.read_csv('dataset/X_test_Final.csv') 

#La funzione ritorna la lisa delle instanze nella quale il modello ha sbagliato
# di una certa percetuale(percentuale=1 -> rmse*1, percentuale=1.5-> rmse*1.5) 
# rispetto al Root Mean Square Error passato come parametro
def extract_instances(rmse, model_name, threshold_error):
    results = pd.read_csv('results/prediction.csv')
    instances_index = np.where(np.abs(results[model_name]-results["Observed"]) > rmse*threshold_error, results["id"],-1)
    instances_index = instances_index[instances_index != -1]
    
    X_test_temp[model_name] = results[model_name]
    X_test_temp["instances_index"] = np.where(np.abs(results[model_name]-results["Observed"]) > rmse*threshold_error, "1", "0")
    fig=px.scatter(X_test_temp, x="Gr_Liv_Area", y="Sale_Price",hover_name="id", hover_data=[model_name], color="instances_index", height=600)
    fig.show()
    fig=px.histogram(X_test_temp, x="Sale_Price",hover_name="id", hover_data=[model_name], color="instances_index", height=600)
    fig.show()
    X_test_temp.drop(model_name, axis=1, inplace=True)
    return X_test_temp.loc[X_test_temp["id"].isin(instances_index)]

# RANDOM FOREST REGRESSOR

In [101]:
rf = RandomForestRegressor()
model_evaluation("Simple RandomForest", rf , X_train, y_train, X_test, y_test, False )

Model Name : Simple RandomForest
RMSE : 23036.14844556519
R2 : 0.9252139155062807
MAE : 15442.10546371684


In [102]:
plot_importance(rf)

In [103]:
instances=extract_instances(25000,"Simple RandomForest", 2)

In [104]:
fig =px.scatter(results, x="Observed", y="Simple RandomForest", hover_data=['id'],color_discrete_sequence=['red'],trendline="ols",trendline_color_override='blue',height=1000, width=1000)
fig.add_shape( type="line", line=dict(dash='dash'),
    x0=results["Observed"].min(), y0=results["Observed"].min(),
    x1=results["Observed"].max(), y1=results["Observed"].max())
fig.show()

## FACCIAMO TUNING DEI PARAMETRI DELLA RANDOM FOREST
### 1 TUNING

In [105]:
## Define Grid 
param_grid = {'n_estimators': [2000,4000,5000], #numero di alberi nella foresta
              'min_samples_split': [2], #numero minimo di campioni in un nodo interno per permettere lo split
              'bootstrap' : [True], #campioni con reinserimento
              'max_depth' : [5,6,7,None], #estensione del'albero, None --> estensione fino a foglie pure
              'max_features': ['sqrt', 'log2', 0.3], #numero di feature su cui sono allenati gli alberi
              } 

grid_search = model_selection.GridSearchCV(rf, #modello utilizzato
                                           param_grid, #griglia dei parametri
                                           scoring=["neg_root_mean_squared_error"], #MSE MEAN SQUARE ERROR
                                           refit ="neg_root_mean_squared_error",
                                           verbose=3,
                                           n_jobs=-1, #multi processor
                                           #cv = 5 <-- default cross validation
                                           return_train_score=True)

best_forest = grid_search.fit(X_train, y_train)#Allena tanti modelli quante sono le combinazioni tra i parametri definiti in param_grid
print('Optimum parameters', best_forest.best_params_)
## show end time
print(datetime.now())

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Optimum parameters {'bootstrap': True, 'max_depth': None, 'max_features': 0.3, 'min_samples_split': 2, 'n_estimators': 2000}
2023-01-12 16:19:42.497295


In [106]:
rf_tuned_1 = best_forest.best_estimator_
model_evaluation("First Tuned RandomForest", rf_tuned_1, X_train, y_train, X_test, y_test, False)

Model Name : First Tuned RandomForest
RMSE : 23311.723044276667
R2 : 0.9234139257369213
MAE : 15236.516459329574


In [107]:
plot_importance(rf_tuned_1)

## Tuned Random Forest - Visualizziamo i valori predetti vs i valori reali osservati

In [90]:
fig =px.scatter(results, x="Observed", y="First Tuned RandomForest", hover_data=['id'],color_discrete_sequence=['red'],trendline="ols",trendline_color_override='blue',height=1000, width=1000)
fig.add_shape( type="line", line=dict(dash='dash'),
    x0=results["Observed"].min(), y0=results["Observed"].min(),
    x1=results["Observed"].max(), y1=results["Observed"].max())
fig.show()

In [46]:
instances=extract_instances(25000,"First Tuned RandomForest", 2)

### 2° TUNING

In [57]:
rf_tuned_2=RandomForestRegressor()

In [62]:
## Define Grid 
param_grid = {'n_estimators': [5000], #numero di alberi nella foresta
              'min_samples_split': [2,4,6], #numero minimo di campioni in un nodo interno per permettere lo split
              'bootstrap' : [True], #campioni con reinserimento
              'max_depth' : [None], #estensione del'albero, None --> estensione fino a foglie pure
              'max_features': [0.3], #numero di feature su cui sono allenati gli alberi
              } 

grid_search = model_selection.GridSearchCV(rf, #modello utilizzato
                                           param_grid, #griglia dei parametri
                                           scoring=["neg_root_mean_squared_error"], #MSE MEAN SQUARE ERROR
                                           refit ="neg_root_mean_squared_error",
                                           verbose=3,
                                           n_jobs=-1, #multi processor
                                           #cv = 5 <-- default cross validation
                                           return_train_score=True)

best_forest2 = grid_search.fit(X_train, y_train)#Allena tanti modelli quante sono le combinazioni tra i parametri definiti in param_grid
print('Optimum parameters', best_forest2.best_params_)
## show end time
print(datetime.now())

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Optimum parameters {'bootstrap': True, 'max_depth': None, 'max_features': 0.3, 'min_samples_split': 2, 'n_estimators': 5000}
2023-01-12 13:41:12.152106


In [63]:
rf_tuned_2 = best_forest2.best_estimator_
model_evaluation("Second Tuned RandomForest", rf_tuned_2, X_train, y_train, X_test, y_test, False)

Model Name : Second Tuned RandomForest
RMSE : 23335.120986301037
R2 : 0.9232601099238098
MAE : 15269.677816242576


In [64]:
plot_importance(rf_tuned_2)

## 2nd Tuned Random Forest - Visualizziamo i valori predetti vs i valori reali osservati

In [89]:
fig =px.scatter(results, x="Observed", y="Second Tuned RandomForest", hover_data=['id'], color_discrete_sequence=['red'],trendline="ols",trendline_color_override='blue',height=1000, width=1000)
fig.add_shape( type="line", line=dict(dash='dash'),
    x0=results["Observed"].min(), y0=results["Observed"].min(),
    x1=results["Observed"].max(), y1=results["Observed"].max())
fig.show()

In [66]:
instances=extract_instances(25000,"Second Tuned RandomForest", 2)

# TRASFORMAZIONE LOG E PROVA DELLA RANDOM FOREST SU DATASET SENZA OUTLIERS

## Trasformazione Logaritmica Sale price

A questo punto come avevamo constatato in fase di visualizzazione della distribuzione di SalePrice proviamo a transformala in scala logaritmica e vediamo se la nostra miglior random forest avrà un RMSE migliore.

In [97]:
rf_log = RandomForestRegressor(bootstrap=True, max_depth=None, min_samples_split=2, n_estimators=5000, max_features=0.3)
model_evaluation("Log RandomForest", rf_log, X_train, y_train, X_test, y_test, True)

Model Name : Log RandomForest
RMSE : 23415.814349723583
R2 : 0.9227284558072336
MAE : 15331.47497393715


## Dataset con outliers

In [68]:
X_train_out = pd.read_csv('dataset/train_data_FE_encoded_With_outliers.csv') #lettura dataset con outliers rimossi con IsolationForest
y_train_out = pd.read_csv('dataset/y_train_With_outliers.csv').squeeze()
X_train_out=X_train_out.drop(["Sale_Price", "id"], axis=1)

In [69]:
rf_out = RandomForestRegressor(bootstrap=True, max_depth=None, min_samples_split=2, n_estimators=5000, max_features=0.3)
model_evaluation("Outliers RandomForest", rf_out, X_train_out, y_train_out, X_test, y_test, False)


Model Name : Outliers RandomForest
RMSE : 23171.01342783914
R2 : 0.9243356829089742
MAE : 15151.767164334224


## Outliers Random Forest - Visualizziamo i valori predetti vs i valori reali osservati

In [98]:
fig =px.scatter(results, x="Observed", y="Outliers RandomForest", hover_data=["id"],color_discrete_sequence=['red'],trendline="ols",trendline_color_override='blue',height=1000, width=1000)
fig.add_shape( type="line", line=dict(dash='dash'),
    x0=results["Observed"].min(), y0=results["Observed"].min(),
    x1=results["Observed"].max(), y1=results["Observed"].max())
fig.show()

In [54]:
instances=extract_instances(25000,"Outliers RandomForest", 2)

In [12]:
results.to_csv('results/prediction.csv', index=False)

In [11]:
#results = results[['id','Observed','Simple RandomForest','First Tuned RandomForest','Second Tuned RandomForest', 'Log RandomForest', 'Outliers RandomForest']]

Il modelli non sembrano essere migliorati molto rispetto alle altre RandomForest.  
Questo ci fa capire che essendo le random forest basate su alberi decisionali indipendenti, addestrati su campioni casuali sia delle osservazioni che delle variabili, non necessitano di una trasformazione della variabile risposta. Grazie al Resampling quindi la random forest risulta essere robusta ai valori outliers e stabile a prescindere dalle trasformazioni delle variabili

# XGBOOST REGRESSOR
Decidiamo di utilizzare un modello di boosting per cercare di migliorare le predizioni sulle istanze con prezzo molto alto e valutiamo se l'effetto è significativo

In [8]:
import xgboost as xgbx
from xgboost.sklearn import XGBRegressor
from xgboost import plot_importance

c:\Users\Marco\Projects\house_prediction\conda-env\lib\site-packages\scipy\__init__.py:146: UserWarning:

A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1

c:\Users\Marco\Projects\house_prediction\conda-env\lib\site-packages\xgboost\compat.py:36: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



# SIMPLE XGBOOST REGRESSOR

In [19]:
xgb_model = XGBRegressor()
model_evaluation("Simple XGBRegressor", xgb_model ,X_train, y_train, X_test, y_test, False)

c:\Users\Marco\Projects\house_prediction\conda-env\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



Model Name : Simple XGBRegressor
RMSE : 23584.30236698373
R2 : 0.9216124434203581
MAE : 15396.101312031036


In [20]:
plot_importance(xgb_model)

In [87]:
fig =px.scatter(results, x="Observed", y="Simple XGBRegressor", hover_data=["id"],color_discrete_sequence=['green'],trendline="ols",trendline_color_override='red',height=1000, width=1000)
fig.add_shape( type="line", line=dict(dash='dash'),
    x0=results["Observed"].min(), y0=results["Observed"].min(),
    x1=results["Observed"].max(), y1=results["Observed"].max())
fig.show()

In [55]:
instances=extract_instances(25000,"Simple XGBRegressor", 2)

# TUNING DEI PARAMETRI DI XGBOOST

In [22]:
xgb_model_tuned = XGBRegressor()

In [23]:

param_grid = { 
              'n_estimators':[5000],#numero di alberi
              'max_depth' : [None, 3], #estensione del'albero, None --> estensione fino a foglie pure
              'colsample_bytree': [0.3], #numero di feature su cui sono allenati gli alberi
              'learning_rate' : [0.01]
              } 

grid_search = model_selection.GridSearchCV(xgb_model_tuned, #modello utilizzato
                                           param_grid, #griglia dei parametri
                                           scoring=["r2","neg_root_mean_squared_error"], #MSE MEAN SQUARE ERROR
                                           refit="neg_root_mean_squared_error", #In the multi-metric setting, you need to set this so that the final model can be fitted to that,
                                                                                #because the best hyper-parameters for the model will be decided based on a single metric only.
                                           verbose=1,
                                           n_jobs=-1, #multi processor
                                           #cv = 5 <-- default cross validation
                                           return_train_score=True)

best_xgboost_model = grid_search.fit(X_train, y_train)#Allena tanti modelli quante sono le combinazioni tra i parametri definiti in param_grid
print('Optimum parameters', best_xgboost_model.best_params_)
print('Best score:', (best_xgboost_model.best_score_))
## show end time
print(datetime.now())

Fitting 5 folds for each of 2 candidates, totalling 10 fits


c:\Users\Marco\Projects\house_prediction\conda-env\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



Optimum parameters {'colsample_bytree': 0.3, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 5000}
Best score: -20197.110230697854
2023-01-12 12:11:37.254267


In [24]:
xgb_model_tuned = best_xgboost_model.best_estimator_
model_evaluation("Tuned XGBRegressor", xgb_model_tuned, X_train, y_train, X_test, y_test, False)

c:\Users\Marco\Projects\house_prediction\conda-env\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



Model Name : Tuned XGBRegressor
RMSE : 20375.812575003853
R2 : 0.9414898874889327
MAE : 13504.83685943895


In [25]:
plot_importance(xgb_model_tuned)

In [86]:
fig =px.scatter(results, x="Observed", y="Tuned XGBRegressor", hover_data=["id"],color_discrete_sequence=['green'],trendline="ols",trendline_color_override='blue',height=1000, width=1000)
fig.add_shape( type="line", line=dict(dash='dash'),
    x0=results["Observed"].min(), y0=results["Observed"].min(),
    x1=results["Observed"].max(), y1=results["Observed"].max())
fig.show()

In [56]:
instances=extract_instances(25000,"Tuned XGBRegressor", 2)

Notiamo che xgb_model_tuned riesce ad avere predizioni migliori sulle case con prezzo più alto rispetto alle random forest allenate in precedenza  
Anche le predizioni delle istanze sotto la curva della distribuzione sembrano essere migliorate rispetto alle random forest
Nella fascia tra i 400 e i 500k le predizioni sono migliorate notevolmente rispetto alla miglior random forest

Dando uno sguardo alle due istanze con ***id 126 e 373*** si nota che queste vengono Overfittate di molto rispetto al prezzo di vendita osservato in una zona dove tutti nostri modelli si comportano molto bene.  
1) Quello che si può notare e che salta all'occhio è che la vendita dell'istanza id=126 è di tipo interfamiliare, quindi in una situazione reale potremmo supporre che sia stata venduta ad un familiare stretto ad un prezzo molto vantaggioso ed è per questo che magari il prezzo è molto inferiore a quello che l'algoritmo di ML(XGBOOST) ha predetto.
2) Nel caso dell'abitazione con id 373, abbiamo una casa che ha una Overall quality pari al massimo della scala disponibile, con un numero di camere e bagni sopra la media, questi fattori potrebbero aver fatto salire molto la previsione del prezzo di vendita sulla base delle istanze nel dataset con le medesime caratteristiche.

In [39]:
xgb_model_tuned_log = best_xgboost_model.best_estimator_
model_evaluation("Tuned XGBRegressor", xgb_model_tuned, X_train, y_train, X_test, y_test, True)

c:\Users\Marco\Projects\house_prediction\conda-env\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



Model Name : Tuned XGBRegressor
RMSE : 20375.812575003853
R2 : 0.9414898874889327
MAE : 13504.83685943895


# KNN - K NEAREST NEIGHBORS

Per curiosità vediamo come si comporta l'algoritmo di KNN sul nostro dataset.

In [27]:
from sklearn.neighbors import KNeighborsRegressor

In [28]:
knn_model_tuned = KNeighborsRegressor()

In [33]:
param_grid = { 
              'n_neighbors': [1,3,5, 7, 9, 12,15,17],
              'weights' : ['uniform','distance'], #pesi uguali o inversamente proporzionali alla distanza
              'metric': ['euclidean', 'manhattan', 'cosine']
              } 

grid_search = model_selection.GridSearchCV(knn_model_tuned, #modello utilizzato
                                           param_grid, #griglia dei parametri
                                           scoring=["neg_root_mean_squared_error"], #MSE MEAN SQUARE ERROR
                                           refit="neg_root_mean_squared_error", #In the multi-metric setting, you need to set this so that the final model can be fitted to that,
                                                                                #because the best hyper-parameters for the model will be decided based on a single metric only.
                                           verbose=1,
                                           n_jobs=-1, #multi processor
                                           #cv = 5 <-- default cross validation
                                           return_train_score=True)

best_knn_model = grid_search.fit(X_train, y_train)#Allena tanti modelli quante sono le combinazioni tra i parametri definiti in param_grid
print('Optimum parameters', best_knn_model.best_params_)
print('Best score:', (best_knn_model.best_score_))
## show end time
print(datetime.now())

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Optimum parameters {'metric': 'manhattan', 'n_neighbors': 12, 'weights': 'distance'}
Best score: -40086.590560627854
2023-01-12 12:16:10.942582


In [34]:
knn_model_tuned = best_knn_model.best_estimator_
model_evaluation("KNN Regressor", knn_model_tuned, X_train, y_train, X_test, y_test, False)

Model Name : KNN Regressor
RMSE : 42378.86315055835
R2 : 0.7468956184877495
MAE : 27200.587571726584


In [99]:
fig =px.scatter(results, x="Observed", y="KNN Regressor", hover_data=["id"],color_discrete_sequence=['green'],trendline="ols",trendline_color_override='blue',height=1000, width=1000)
fig.add_shape( type="line", line=dict(dash='dash'),
    x0=results["Observed"].min(), y0=results["Observed"].min(),
    x1=results["Observed"].max(), y1=results["Observed"].max())
fig.show()

In [67]:
instances=extract_instances(25000,"KNN Regressor", 2) #visualizzo le istanze in cui il prezzo predetto differisce del 100%(in negativo o positivo)
                                                           #rispetto al RMSE

In [ ]:
import dill

def save_session() :
    dill.dump_session('data_prediction.db')
    
def load_session():
    dill.load_session('data_prediction.db')

In [ ]:
#save_session()

In [ ]:
#load_session()